In [1]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
# partly from https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
RUG = pd.read_pickle("../obfuscated_data.pkl")
RUG.interpolate(method='linear', inplace=True)#, limit=20)
RUG = RUG[::10]
dfs = [RUG.filter([i]) for i in RUG]

In [5]:
dfs = [i.dropna() for i in dfs]

In [6]:
# RUG

# Timeseries LSTM

In [7]:
def func(df, name):

    df2 = df.copy()

    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(df2)

    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]


    # convert an array of values into a dataset matrix
    def create_dataset(dataset, look_back=3):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return np.array(dataX), np.array(dataY)
    
    # reshape into X=t and Y=t+1
    look_back = 3
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    
    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2, min_lr=0.001, verbose=2)

    # callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)


    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.fit(trainX, trainY, epochs=100, verbose=2, callbacks=[early_stopping, reduce_lr], batch_size=256)

    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    # calculate root mean squared error
    rmse_train = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    # print('Train Score: %.2f RMSE' % (trainScore))
    rmse_test = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    # print('Test Score: %.2f RMSE' % (testScore))

    mae_train = tf.keras.metrics.mean_absolute_error(trainY[0], trainPredict[:,0]).numpy()
    mae_test = tf.keras.metrics.mean_absolute_error(testY[0], testPredict[:,0]).numpy()

    mape_train = tf.keras.metrics.mean_absolute_percentage_error(trainY[0], trainPredict[:,0]).numpy()
    mape_test = tf.keras.metrics.mean_absolute_percentage_error(testY[0], testPredict[:,0]).numpy()

    return (name, (rmse_train, rmse_test, mae_train, mae_test, mape_train, mape_test))

In [8]:
results = []

for name, df in zip([i for i in RUG], dfs):
    print(name)
    
    if df.isnull().values.any():
        print("NaN")
        results.append((name, (np.nan, np.nan)))
        print("-------------")
        continue

    r = func(df, name)
    print("-------------")
    results.append(r)

Location 1 - flow
Epoch 1/100
803/803 - 13s - loss: 0.0291 - mse: 0.0291 - lr: 0.0010 - 13s/epoch - 16ms/step
Epoch 2/100
803/803 - 4s - loss: 0.0044 - mse: 0.0044 - lr: 0.0010 - 4s/epoch - 4ms/step
Epoch 3/100
803/803 - 3s - loss: 0.0038 - mse: 0.0038 - lr: 0.0010 - 3s/epoch - 4ms/step
Epoch 4/100
803/803 - 3s - loss: 0.0036 - mse: 0.0036 - lr: 0.0010 - 3s/epoch - 4ms/step
Epoch 5/100
803/803 - 3s - loss: 0.0033 - mse: 0.0033 - lr: 0.0010 - 3s/epoch - 4ms/step
Epoch 6/100
803/803 - 3s - loss: 0.0031 - mse: 0.0031 - lr: 0.0010 - 3s/epoch - 4ms/step
Epoch 7/100
803/803 - 3s - loss: 0.0031 - mse: 0.0031 - lr: 0.0010 - 3s/epoch - 4ms/step
Epoch 8/100
803/803 - 3s - loss: 0.0031 - mse: 0.0031 - lr: 0.0010 - 3s/epoch - 4ms/step
Epoch 9/100
803/803 - 3s - loss: 0.0031 - mse: 0.0031 - lr: 0.0010 - 3s/epoch - 4ms/step
Epoch 10/100
803/803 - 3s - loss: 0.0031 - mse: 0.0031 - lr: 0.0010 - 3s/epoch - 4ms/step
Epoch 11/100
803/803 - 3s - loss: 0.0031 - mse: 0.0031 - lr: 0.0010 - 3s/epoch - 4ms/ste

In [16]:
# for i in results:
#     print(i)

In [9]:
lstm_results = [[i[0], i[1][0], i[1][1], i[1][2], i[1][3], i[1][4], i[1][5]] for i in results]

lstm_results = pd.DataFrame(lstm_results)

lstm_results.columns = ["Location", "Train RMSE", "Test RMSE", "Train MAE", "Test MAE", "Train MAPE", "Test MAPE"]
lstm_results.set_index("Location", inplace=True)
lstm_results.sort_index(inplace=True)
lstm_results = lstm_results.astype(float).round(3)

display(lstm_results)

# print(lstm_results.round(3).to_latex())

,Train RMSE,Test RMSE,Train MAE,Test MAE,Train MAPE,Test MAPE
Location,,,,,,
Location 1 - flow,26.112,26.388,13.864,16.983,7.138407e+07,6.678130e+02
Location 10 - flow,17.722,20.980,5.544,6.381,4.492171e+03,2.056085e+03
Location 11 - flow,5.409,1.962,0.918,0.217,1.727385e+08,1.529896e+08
Location 11 - head,25.623,0.408,0.411,0.253,5.350000e-01,4.710000e-01
Location 12 - head,0.697,0.588,0.334,0.303,5.050000e-01,4.560000e-01
Location 2 - consumption,0.283,0.411,0.084,0.233,1.205500e+01,6.532252e+04
Location 3 - consumption,0.406,0.470,0.132,0.206,4.999969e+06,5.986138e+06
Location 4 - consumption,0.356,0.687,0.097,0.305,2.573000e+00,2.182848e+04
Location 5 - consumption,0.207,0.406,0.078,0.280,5.360159e+04,1.877709e+06


In [10]:
cols = pd.MultiIndex.from_product([['Train', 'Test'], ['RMSE', 'MAE', 'MAPE']])
lstm_results.columns = cols
print(lstm_results.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
 & \multicolumn{3}{r}{Train} & \multicolumn{3}{r}{Test} \\
 & RMSE & MAE & MAPE & RMSE & MAE & MAPE \\
Location &  &  &  &  &  &  \\
\midrule
Location 1 - flow & 26.112000 & 26.388000 & 13.864000 & 16.983000 & 71384072.000000 & 667.813000 \\
Location 10 - flow & 17.722000 & 20.980000 & 5.544000 & 6.381000 & 4492.171000 & 2056.085000 \\
Location 11 - flow & 5.409000 & 1.962000 & 0.918000 & 0.217000 & 172738496.000000 & 152989632.000000 \\
Location 11 - head & 25.623000 & 0.408000 & 0.411000 & 0.253000 & 0.535000 & 0.471000 \\
Location 12 - head & 0.697000 & 0.588000 & 0.334000 & 0.303000 & 0.505000 & 0.456000 \\
Location 2 - consumption & 0.283000 & 0.411000 & 0.084000 & 0.233000 & 12.055000 & 65322.523000 \\
Location 3 - consumption & 0.406000 & 0.470000 & 0.132000 & 0.206000 & 4999969.000000 & 5986138.500000 \\
Location 4 - consumption & 0.356000 & 0.687000 & 0.097000 & 0.305000 & 2.573000 & 21828.484000 \\
Location 5 - consumption & 0.207000 & 0.4060